In [1]:
from agential.cog.agent.critic import CriticAgent
from agential.cog.agent.critic_old import CriticAgent as OldCriticAgent
from langchain_community.utilities.google_search import GoogleSearchAPIWrapper

from langchain_community.chat_models.openai import ChatOpenAI
from agential.cog.prompts.critic import (
    # QA.
    CRITIC_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT,
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_FEVER,
    FEVER_FEWSHOT_EXAMPLES_COT,
    FEVER_FEWSHOT_EXAMPLES_DIRECT,
    FEVER_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_FEVER,
    FEVER_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_AMBIGNQ,
    AMBIGNQ_FEWSHOT_EXAMPLES_COT,
    AMBIGNQ_FEWSHOT_EXAMPLES_DIRECT,
    AMBIGNQ_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_AMBIGNQ,
    AMBIGNQ_FEWSHOT_EXAMPLES_CRITIC,

    CRITIC_INSTRUCTION_TRIVIAQA,
    TRIVIAQA_FEWSHOT_EXAMPLES_COT,
    TRIVIAQA_FEWSHOT_EXAMPLES_DIRECT,
    TRIVIAQA_FEWSHOT_EXAMPLES_REACT,
    CRITIC_CRITIQUE_INSTRUCTION_TRIVIAQA,
    TRIVIAQA_FEWSHOT_EXAMPLES_CRITIC,

    # Math.
    CRITIC_POT_INSTRUCTION_GSM8K,
    GSM8K_FEWSHOT_EXAMPLES_POT,
    CRITIC_CRITIQUE_INSTRUCTION_GSM8K,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K,
    GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,

    CRITIC_POT_INSTRUCTION_SVAMP,
    SVAMP_FEWSHOT_EXAMPLES_POT,
    CRITIC_CRITIQUE_INSTRUCTION_SVAMP,
    SVAMP_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_SVAMP,
    SVAMP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,

    CRITIC_POT_INSTRUCTION_TABMWP,
    TABMWP_FEWSHOT_EXAMPLES_POT,
    CRITIC_CRITIQUE_INSTRUCTION_TABMWP,
    TABMWP_FEWSHOT_EXAMPLES_CRITIC,
    CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_TABMWP,
    TABMWP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
)

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings("ignore")

llm = ChatOpenAI()
search = GoogleSearchAPIWrapper()

# QA

In [ ]:
agent = CriticAgent(
    llm=llm, mode="qa", search=search
)

In [ ]:
use_search_tool = False

In [ ]:
# HotpotQA
question = 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring'
out = agent.generate(
    question=question,
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,  # HOTPOTQA_FEWSHOT_EXAMPLES_DIRECT, HOTPOTQA_FEWSHOT_EXAMPLES_REACT
    prompt=CRITIC_INSTRUCTION_HOTPOTQA,
    critique_examples=HOTPOTQA_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_HOTPOTQA,
    use_search_tool=use_search_tool
)

In [ ]:
out

In [ ]:
# FEVER
question = "Nikolaj Coster-Waldau worked with the Fox Broadcasting Company."
out = agent.generate(
    question=question,
    examples=FEVER_FEWSHOT_EXAMPLES_COT,  # FEVER_FEWSHOT_EXAMPLES_DIRECT, FEVER_FEWSHOT_EXAMPLES_REACT
    prompt=CRITIC_INSTRUCTION_FEVER,
    critique_examples=FEVER_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_FEVER,
    use_search_tool=use_search_tool
)

In [ ]:
out

In [ ]:
# AmbigNQ
question = "When did the simpsons first air on television?"
out = agent.generate(
    question=question,
    examples=AMBIGNQ_FEWSHOT_EXAMPLES_COT,  # AMBIGNQ_FEWSHOT_EXAMPLES_DIRECT, AMBIGNQ_FEWSHOT_EXAMPLES_REACT
    prompt=CRITIC_INSTRUCTION_AMBIGNQ,
    critique_examples=AMBIGNQ_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_AMBIGNQ,
    use_search_tool=use_search_tool
)

In [ ]:
out

In [ ]:
# TriviaQA
question = "Which American-born Sinclair won the Nobel Prize for Literature in 1930?"
out = agent.generate(
    question=question,
    examples=TRIVIAQA_FEWSHOT_EXAMPLES_COT,  # TRIVIAQA_FEWSHOT_EXAMPLES_DIRECT, TRIVIAQA_FEWSHOT_EXAMPLES_REACT
    prompt=CRITIC_INSTRUCTION_TRIVIAQA,
    critique_examples=TRIVIAQA_FEWSHOT_EXAMPLES_CRITIC,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_TRIVIAQA,
    use_search_tool=use_search_tool
)

In [ ]:
out

# Math

In [2]:
agent = CriticAgent(
    llm=llm, mode="math"
)

In [3]:
use_interpreter_tool = False

In [4]:
# GSM8k
question = "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with 4933828. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
out = agent.generate(
    question=question,
    examples=GSM8K_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_GSM8K,
    critique_examples=GSM8K_FEWSHOT_EXAMPLES_CRITIC if use_interpreter_tool else GSM8K_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_GSM8K if use_interpreter_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_GSM8K,
    use_interpreter_tool=use_interpreter_tool
)

<PROMPT AGENT===========================================================>
# Write Python Code to solve the following questions. Store your result as a variable named 'answer'.

Question: Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
# Python code, return answer
total_eggs = 16
eaten_eggs = 3
baked_eggs = 4
sold_eggs = total_eggs - eaten_eggs - baked_eggs
dollars_per_egg = 2
answer = sold_eggs * dollars_per_egg

Question: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
# Python code, return answer
bolts_of_blue_fiber = 2
bolts_of_white_fiber = num_of_blue_fiber / 2
answer = bolts_of_blue_fiber + bolts_of_white_fiber

Question: Josh decides to try flipping a house.  He buys a house for $80,000 and th

In [5]:
out

[{'code': 'total_eggs = 16\neaten_eggs = 3\nbaked_eggs = 4933828\nsold_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\nanswer = sold_eggs * dollars_per_egg',
  'critique': "The code is incorrect. The number of eggs baked by Janet for muffins (4933828) seems to be unreasonably high, which results in a negative value for `sold_eggs` and ultimately an incorrect calculation for the amount of money she makes at the farmers' market."},
 {'code': 'total_eggs = 16\neaten_eggs = 3\nbaked_eggs = 12  # Assume Janet bakes 12 eggs for muffins daily\nsold_eggs = total_eggs - eaten_eggs - baked_eggs\ndollars_per_egg = 2\nanswer = sold_eggs * dollars_per_egg',
  'critique': '< CORRECT>'}]

In [6]:
# SVAMP
question = "There are 87 oranges and 290 bananas in Philip's collection. If the bananas are organized into 2 groups and oranges are organized into 93 groups"
out = agent.generate(
    question=question,
    examples=SVAMP_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_SVAMP,
    critique_examples=SVAMP_FEWSHOT_EXAMPLES_CRITIC if use_interpreter_tool else SVAMP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_SVAMP if use_interpreter_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_SVAMP,
    use_interpreter_tool=use_interpreter_tool
)

<PROMPT AGENT===========================================================>
Read the following passages to answer questions with Python code, store the result as a 'answer' variable:

Question: James bought 93 red and 10 blue stickers, he used 31 red sticker on his fridge and 7 blue stickers on his laptop. How many red stickers does James have?
# Python code, return answer
original_red_stickers = 93
used_red_stickers = 31
answer = original_red_stickers - used_red_stickers

Question: Allen went to supermarket to buy eggs, each egg costs 80 dollars, if the discount is 29 dollars. How much do you have to pay to buy for each egg?
# Python code, return answer
original_egg_price_in_dollars = 80
discount_dollars = 29
answer = original_egg_price_in_dollars - discount_dollars

Question: Dianna collects both cases and books. He bought 22 cases and 5 books from the store. Now he has 57 cases and 25 books. How many books did danny have at first?
# Python code, return answer
num_books_bought_at_store

In [ ]:
out

In [ ]:
# TabMWP
question = """Read the following table regarding "Bowling Scores" and then write Python code to answer a question:

Name | Score
Amanda | 117
Sam | 236
Irma | 144
Mike | 164

Question: Some friends went bowling and kept track of their scores. How many more points did Mike score than Irma?"""
out = agent.generate(
    question=question,
    examples=TABMWP_FEWSHOT_EXAMPLES_POT,
    prompt=CRITIC_POT_INSTRUCTION_TABMWP,
    critique_examples=TABMWP_FEWSHOT_EXAMPLES_CRITIC if use_interpreter_tool else TABMWP_FEWSHOT_EXAMPLES_CRITIC_NO_TOOL,
    critique_prompt=CRITIC_CRITIQUE_INSTRUCTION_TABMWP if use_interpreter_tool else CRITIC_CRITIQUE_NO_TOOL_INSTRUCTION_TABMWP,
    use_interpreter_tool=use_interpreter_tool
)

In [ ]:
out